## Cross in Tray 

In [1]:
using Pkg
Pkg.activate("../../.")
using Globtim
using DynamicPolynomials, DataFrames
using ProgressLogging

  Activating project at `~/globtim`


### Questions to fix:
 - [ ]: Why are we increasing the degree of the approximant and why are we using such a large number of points? The constructor only specifies a few. 
 - [x]: Filter critical points.

In [4]:
n = 2
d = 8 # Initial Degree 
SMPL = 10 # Number of samples
center = [0.0, 0.0]
TR = test_input(f,
    dim=n,
    center=[0.0, 0.0],
    GN=SMPL,
    sample_range=scale_factor, 
    tolerance=tol_l2,
)
pol_cheb = Constructor(TR, d, basis=:chebyshev)
pol_lege = Constructor(TR, d, basis=:legendre);

UndefVarError: UndefVarError: `scale_factor` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [5]:
@polyvar(x[1:n]) # Define polynomial ring 

(Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}[x₁, x₂],)

In [6]:
real_pts_cheb = solve_polynomial_system(x, TR.dim, pol_cheb.degree, pol_cheb.coeffs; basis=:chebyshev, bigint=true)
real_pts_lege = solve_polynomial_system(x, TR.dim, pol_lege.degree, pol_lege.coeffs; basis=:legendre, bigint=true)

UndefVarError: UndefVarError: `TR` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
# Usage example:
df_cheb = process_crit_pts(real_pts_cheb, f, TR)
df_lege = process_crit_pts(real_pts_lege, f, TR)


=== Starting Critical Points Processing (dimension: 2) ===
Processed 49 points (0.035s)

=== Starting Critical Points Processing (dimension: 2) ===
Processed 49 points (0.0s)


Row,x1,x2,z
,Float64,Float64,Float64
1,9.26292,9.26292,-10.0763
2,6.89455,9.25401,-12.0875
3,-9.89628e-17,9.26948,-0.341423
4,-6.89455,9.25401,-12.0875
5,-9.26292,9.26292,-10.0763
6,-4.06725,9.26211,-13.0106
7,4.06725,9.26211,-13.0106
8,9.25401,6.89455,-12.0875
9,6.88499,6.88499,-14.4187


In [7]:
using Optim
df_cheb, df_min_cheb = analyze_critical_points(f, df_cheb, TR, tol_dist=0.01)
df_lege, df_min_lege = analyze_critical_points(f, df_lege, TR, tol_dist=0.01)

Processing point 1 of 49
Optimization has converged: ✓
Processing point 2 of 49
Optimization has converged: ✓
Processing point 3 of 49
Optimization has converged: ✓
Processing point 4 of 49
Optimization has converged: ✓
Processing point 5 of 49
Optimization has converged: ✓
Processing point 6 of 49
Optimization has converged: ✓
Processing point 7 of 49
Optimization has converged: ✓
Processing point 8 of 49
Optimization has converged: ✓
Processing point 9 of 49
Optimization has converged: ✓
Processing point 10 of 49
Optimization has converged: ✓
Processing point 11 of 49
Optimization has converged: ✓
Processing point 12 of 49
Optimization has converged: ✓
Processing point 13 of 49
Optimization has converged: ✓
Processing point 14 of 49
Optimization has converged: ✓
Processing point 15 of 49
Optimization has converged: ✓
Processing point 16 of 49
Optimization has converged: ✗
Processing point 17 of 49
Optimization has converged: ✓
Processing point 18 of 49
Optimization has converged: ✓
P

(49×8 DataFrame
 Row │ x1            x2        z           y1            y2           close  s ⋯
     │ Float64       Float64   Float64     Float64       Float64      Bool   F ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │  9.26292       9.26292  -10.0763      -10.7742      -10.7742   false    ⋯
   2 │  6.89455       9.25401  -12.0875       10.7421       -7.67107  false
   3 │ -9.89628e-17   9.26948   -0.341423    -20.1316       -7.74024  false
   4 │ -6.89455       9.25401  -12.0875      -10.7421       -7.67107  false
   5 │ -9.26292       9.26292  -10.0763       10.7742      -10.7742   false    ⋯
   6 │ -4.06725       9.26211  -13.0106       -4.41908       1.47062  false
   7 │  4.06725       9.26211  -13.0106        4.41908       1.47062  false
   8 │  9.25401       6.89455  -12.0875       -7.67107      10.7421   false
  ⋮  │      ⋮           ⋮          ⋮            ⋮             ⋮         ⋮      ⋱
  43 │  9.26211       4.06725  -13.0106   

In [8]:
using GLMakie

In [9]:
plot_polyapprox_rotate(pol_lege, TR, df_lege, df_min_lege)

In [10]:
plot_polyapprox_levelset(pol_cheb, TR, df_cheb, df_min_cheb)

In [11]:
# GLMakie.closeall()